In [21]:
%matplotlib inline
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


In [1]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import warnings; warnings.simplefilter('ignore')

def weighted_rating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def filter_keywords(x, s):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

cols = ['id', 'movieId', 'title', 'genres', 'description', 'keywords', \
        'cast', 'director', 'spoken_languages', 'production_companies',\
        'production_countries', 'popularity', 'year', 'vote_average',\
         'vote_count', 'wr']

def read_dataset(metadata_path, links_small_path, credits_path,keywords_path):
    meta = pd.read_csv(metadata_path)
    links_small = pd.read_csv(links_small_path)
    credits = pd.read_csv(credits_path)
    keywords = pd.read_csv(keywords_path)
    return meta, links_small, credits, keywords

def cal_weighted_rating(meta, percentile=0.95):
    vote_counts = meta[meta['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = meta[meta['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    meta = meta[(meta['vote_count'].notnull()) \
                     & (meta['vote_average'].notnull())]
    meta['vote_count'] = meta['vote_count'].astype('int')
    meta['vote_average'] = meta['vote_average'].astype('int')
    meta['wr'] = meta.apply(lambda x: weighted_rating(x, m, C), axis=1)

    return meta
    
def movie_feature(metadata_path, links_small_path, credits_path,keywords_path, \
                  percentile=0.95, more_weight_on = None, \
                    stemmer = SnowballStemmer('english'), cols=cols):
    # read dataset
    meta, links_small, credits, keywords = read_dataset(metadata_path, \
        links_small_path, credits_path,keywords_path)

    # change type + drop
    links_small = links_small[links_small['tmdbId'].notnull()]
    links_small['tmdbId'] = links_small['tmdbId'].astype('int')

    meta = meta.drop([19730, 29503, 35587])
    meta['popularity'] = meta[meta['popularity'].notnull()]['popularity'].astype('float')
    meta['id'] = meta['id'].astype('int')
    meta['year'] = pd.to_datetime(meta['release_date'], errors='coerce').apply(\
        lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
    meta = meta[meta.production_companies.notnull()]
    keywords['id'] = keywords['id'].astype('int')
    credits['id'] = credits['id'].astype('int')

    # calcualte weighted rating for movies
    meta = cal_weighted_rating(meta)

    # merge meta + link small => create a smaller dataset for recommend
    smd = meta[meta['id'].isin(links_small['tmdbId'])]

    # create description feature
    smd['tagline'] = smd['tagline'].fillna('')
    smd['description'] = smd['overview'] + smd['tagline']
    smd['description'] = smd['description'].fillna('')
    
    # merge credit + keywords + links_small
    smd = smd.merge(credits, on='id')
    smd = smd.merge(keywords, on='id')
    smd = smd.merge(links_small, left_on='id', right_on='tmdbId')

    # feature engineering
    literal_features = ['cast', 'spoken_languages', 'genres', 'keywords',\
                        'production_companies', 'production_countries']
    for fearture in literal_features:
        smd[fearture] = smd[fearture].apply(literal_eval)
        smd[fearture] = smd[fearture].apply(lambda x: [i['name'] \
                                            for i in x] if isinstance(x, list) else [])
    smd['crew'] = smd['crew'].apply(literal_eval)
    smd['director'] = smd['crew'].apply(get_director)

    # top 3 actors
    smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
   
    # Strip Spaces and Convert to Lowercase 
    smd['cast'] = smd['cast'].apply(lambda x: [str.lower(\
        i.replace(" ", "")) for i in x])
    smd['director'] = smd['director'].astype('str').apply(\
        lambda x: str.lower(x.replace(" ", "")))
    if more_weight_on:
        smd[more_weight_on] = smd[more_weight_on].apply(lambda x: [x,x,x])
    
    # choose keywords appear more than once + stemming
    s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
    s = s.value_counts()
    s = s[s > 1]
    smd['keywords'] = smd['keywords'].apply(lambda x: filter_keywords(x, s))
    smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
    smd['keywords'] = smd['keywords'].apply(lambda x: \
                                            [str.lower(i.replace(" ", "")) for i in x])

    return smd[cols]
    


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
credits_ = r'/kaggle/input/the-movies-dataset/credits.csv'
keywords = r'/kaggle/input/the-movies-dataset/keywords.csv'
links = r'/kaggle/input/the-movies-dataset/links_small.csv'
movies_metadata = r'/kaggle/input/the-movies-dataset/movies_metadata.csv'
smd = movie_feature(movies_metadata, links, credits_, keywords, more_weight_on='director')
# print(smd.columns)

In [11]:
smd["description"] = smd["description"] + smd['keywords'].apply(lambda x: " ".join(map(str, x))) + smd['production_companies'].apply(lambda x: " ".join(map(str, x)))

['jealousi',
 'toy',
 'boy',
 'friendship',
 'friend',
 'rivalri',
 'boynextdoor',
 'newtoy',
 'toycomestolif']

In [15]:
smd = smd[['id', 'movieId', 'title', 'description']]
smd = smd.drop_duplicates()

In [16]:
smd

,id,movieId,title,description
0,862,1,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,2,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,3,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,4,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,5,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...,...
9214,159550,161944,The Last Brickmaker in America,A man must cope with the loss of his wife and ...
9215,392572,162542,Rustom,"Rustom Pavri, an honourable officer of the Ind..."
9216,402672,162672,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj..."
9217,315011,163056,Shin Godzilla,From the mind behind Evangelion comes a hit la...


### TF-IDF

In [17]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])
tfidf_matrix

<9082x358256 sparse matrix of type '<class 'numpy.float64'>'
	with 736021 stored elements in Compressed Sparse Row format>

In [23]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [24]:
cosine_sim.shape

(9082, 9082)

In [25]:
smd = smd.reset_index()
smd = smd.drop(columns='index')
movie_ids = smd['movieId']
indices = pd.Series(smd.index, index=smd['movieId'])

## Evaluate

In [26]:
train_df = pd.read_csv('/kaggle/input/train-set-from-ratings/train_set.csv')

In [27]:
train_df

,userId,movieId,rating,timestamp
0,431,2863,4.0,1165548515
1,571,7173,2.0,1334343358
2,77,223,4.5,1163004353
3,580,1032,4.0,1165291033
4,624,1221,5.0,1019124147
...,...,...,...,...
74998,547,5810,3.0,1415444349
74999,418,1835,4.0,1132180632
75000,5,33679,4.0,1163374517
75001,358,905,5.0,957479957


In [28]:
def get_recommendations(user_id, cosine_sim, n_rec = 10):
    movies_id = train_df[train_df.userId == user_id].movieId 
    recommended = []
    for movie in movies_id:
        try:
            idx = indices[movie]
        except:
            # In case movie is not available in cosine_sim (or not in metadata)
            print("Something went wrong at movieId " + str(movie))
        else:
            # top 10 most similar
            sim_scores = list(enumerate(cosine_sim[idx]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = sim_scores[1:11]
        
            for movie_rec in sim_scores:
                # change index -> movieId
                movie_recommended = (movie_ids.iloc[movie_rec[0]], movie_rec[1])
                recommended.append(movie_recommended)
    # create a DataFrame to store all movies recommended            
    recommended = pd.DataFrame(recommended)  
    recommended.columns = ['movieId', 'sim']
    recommended = recommended.sort_values('sim')[::-1].head(n_rec)
    recommended['userId'] = user_id
    return recommended.drop(columns='sim')

In [29]:
def full_recommend(df, cosine_sim):
    rec_df = pd.DataFrame()
    for user in df.userId.value_counts().index:
        rec_df = pd.concat([rec_df, get_recommendations(user, cosine_sim)])
    return rec_df

In [30]:
recommended_df = full_recommend(train_df, cosine_sim)

Something went wrong at movieId 96075
Something went wrong at movieId 7669
Something went wrong at movieId 26649
Something went wrong at movieId 150856
Something went wrong at movieId 720
Something went wrong at movieId 2851
Something went wrong at movieId 4051
Something went wrong at movieId 720
Something went wrong at movieId 55207
Something went wrong at movieId 108583
Something went wrong at movieId 7502
Something went wrong at movieId 73759
Something went wrong at movieId 27611
Something went wrong at movieId 77359
Something went wrong at movieId 100450
Something went wrong at movieId 94466
Something went wrong at movieId 7502
Something went wrong at movieId 27611
Something went wrong at movieId 720
Something went wrong at movieId 2851
Something went wrong at movieId 27611
Something went wrong at movieId 106642
Something went wrong at movieId 31193
Something went wrong at movieId 108979
Something went wrong at movieId 4207
Something went wrong at movieId 7669
Something went wrong 

In [31]:
recommended_df

,movieId,userId
5770,50742,547
14480,26712,547
15530,3077,547
15531,50742,547
5771,26712,547
...,...,...
142,35836,484
130,3450,484
10,200,484
1,6975,484


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

def split_data(rating_df):
    train_df, test_df = train_test_split(rating_df, \
                                         test_size=0.2, stratify=rating_df['userId'], \
                                        random_state=42)
    return train_df, test_df

def check_movieId(pred_df, val_df):
    result = pred_df['movieId'].isin(val_df[val_df['userId'] == \
                                   pred_df['userId'].iloc[0]]['movieId'])
    return result.reset_index(drop=True)

def evaluate(pred_df, val_df):
    """ Proportion of movies recommended that were actually watched by users

    Args:
    pred_df: dataframe, 2 columns: userId, movieId
    val_df: dataframe, 2 columns: userId, movieId

    Returns:
    
    """
    result = pred_df.groupby('userId').apply(lambda x: check_movieId(x, val_df))
    result = result.groupby('userId').sum() / result.groupby('userId').count()

    return result.mean().mean()


    

In [33]:
testset = pd.read_csv(r'/kaggle/input/train-set-from-ratings/test_set.csv')

In [34]:
testset = testset[['userId', 'movieId']]
testset

,userId,movieId
0,302,593
1,191,110
2,457,5337
3,239,1042
4,292,5377
...,...,...
24996,353,34319
24997,428,743
24998,595,1097
24999,664,61248


In [35]:
evaluate(recommended_df, testset)

0.06408345752608048

### Count Vectorize

In [36]:
count_vec = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_vec_matrix = count_vec.fit_transform(smd['description'])
count_vec_matrix

<9082x358256 sparse matrix of type '<class 'numpy.int64'>'
	with 736021 stored elements in Compressed Sparse Row format>

In [37]:
count_cosine_sim = cosine_similarity(count_vec_matrix, count_vec_matrix)
count_cosine_sim

array([[1.        , 0.00944448, 0.        , ..., 0.        , 0.        ,
        0.00862807],
       [0.00944448, 1.        , 0.02035208, ..., 0.0105462 , 0.01801802,
        0.01646049],
       [0.        , 0.02035208, 1.        , ..., 0.01191236, 0.01017604,
        0.        ],
       ...,
       [0.        , 0.0105462 , 0.01191236, ..., 1.        , 0.0105462 ,
        0.        ],
       [0.        , 0.01801802, 0.01017604, ..., 0.0105462 , 1.        ,
        0.00823024],
       [0.00862807, 0.01646049, 0.        , ..., 0.        , 0.00823024,
        1.        ]])

In [38]:
# compare with cosine_sim (tf-idf)
cosine_sim

array([[1.00000000e+00, 5.15607359e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 4.07623723e-03],
       [5.15607359e-03, 1.00000000e+00, 1.21721276e-02, ...,
        5.83638692e-04, 1.92425346e-03, 3.07377942e-03],
       [0.00000000e+00, 1.21721276e-02, 1.00000000e+00, ...,
        1.53066359e-03, 1.86281887e-03, 0.00000000e+00],
       ...,
       [0.00000000e+00, 5.83638692e-04, 1.53066359e-03, ...,
        1.00000000e+00, 5.80586716e-04, 0.00000000e+00],
       [0.00000000e+00, 1.92425346e-03, 1.86281887e-03, ...,
        5.80586716e-04, 1.00000000e+00, 1.25417180e-03],
       [4.07623723e-03, 3.07377942e-03, 0.00000000e+00, ...,
        0.00000000e+00, 1.25417180e-03, 1.00000000e+00]])

In [39]:
count_vectorized_recommended_df = full_recommend(train_df, count_cosine_sim)

Something went wrong at movieId 96075
Something went wrong at movieId 7669
Something went wrong at movieId 26649
Something went wrong at movieId 150856
Something went wrong at movieId 720
Something went wrong at movieId 2851
Something went wrong at movieId 4051
Something went wrong at movieId 720
Something went wrong at movieId 55207
Something went wrong at movieId 108583
Something went wrong at movieId 7502
Something went wrong at movieId 73759
Something went wrong at movieId 27611
Something went wrong at movieId 77359
Something went wrong at movieId 100450
Something went wrong at movieId 94466
Something went wrong at movieId 7502
Something went wrong at movieId 27611
Something went wrong at movieId 720
Something went wrong at movieId 2851
Something went wrong at movieId 27611
Something went wrong at movieId 106642
Something went wrong at movieId 31193
Something went wrong at movieId 108979
Something went wrong at movieId 4207
Something went wrong at movieId 7669
Something went wrong 

In [40]:
evaluate(count_vectorized_recommended_df, testset)

0.05275707898658718